## 8. Robustness and Performance

### 65 Take Advantage of Each Block in `try`/`except`/`else`/`finally`

In [1]:
import logging

In [2]:
def try_finally_example(filename):
    print('* Opening file')
    handle = open(filename, encoding='utf-8') # May raise OSError
    try:
        print('* Reading data')
        return handle.read()  # May raise UnicodeDecodeError
    finally:
        print('* Calling close()')
        handle.close()        # Always runs after try block

In [3]:
filename = 'random_data.txt'

with open(filename, 'wb') as f:
    f.write(b'\xf1\xf2\xf3\xf4\xf5')  # Invalid utf-8

try:
    data = try_finally_example(filename)
    # This should not be reached.
    import sys
    sys.exit(1)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-3-4e15b121efbf>", line 7, in <module>
    data = try_finally_example(filename)
  File "<ipython-input-2-c65eccd18c13>", line 6, in try_finally_example
    return handle.read()  # May raise UnicodeDecodeError
  File "/usr/local/lib/python3.8/codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 0: invalid continuation byte


* Opening file
* Reading data
* Calling close()


In [4]:
try:
    try_finally_example('does_not_exist.txt')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-4-68fef72a45b6>", line 2, in <module>
    try_finally_example('does_not_exist.txt')
  File "<ipython-input-2-c65eccd18c13>", line 3, in try_finally_example
    handle = open(filename, encoding='utf-8') # May raise OSError
FileNotFoundError: [Errno 2] No such file or directory: 'does_not_exist.txt'


* Opening file


In [5]:
import json

In [6]:
def load_json_key(data, key):
    try:
        print('* Loading JSON data')
        result_dict = json.loads(data)  # May raise ValueError
    except ValueError as e:
        print('* Handling ValueError')
        raise KeyError(key) from e
    else:
        print('* Looking up key')
        return result_dict[key]         # May raise KeyError

In [7]:
assert load_json_key('{"foo": "bar"}', 'foo') == 'bar'

* Loading JSON data
* Looking up key


In [8]:
try:
    load_json_key('{"foo": bad payload', 'foo')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-6-89f4737e3fac>", line 4, in load_json_key
    result_dict = json.loads(data)  # May raise ValueError
  File "/usr/local/lib/python3.8/json/__init__.py", line 357, in loads
    return _default_decoder.decode(s)
  File "/usr/local/lib/python3.8/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/local/lib/python3.8/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 9 (char 8)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "<ipython-input-8-f3d18efb877c>", line 2, in <module>
    load_json_key('{"foo": bad payload', 'foo')
  File "<ipython-input-6-89f4737e3fac>", line 7, in load_json_key
    raise KeyError(key) from e
KeyError: 'foo'


* Loading JSON data
* Handling ValueError


In [9]:
try:
    load_json_key('{"foo": "bar"}', 'does not exist')
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-9-69ce2b80d504>", line 2, in <module>
    load_json_key('{"foo": "bar"}', 'does not exist')
  File "<ipython-input-6-89f4737e3fac>", line 10, in load_json_key
    return result_dict[key]         # May raise KeyError
KeyError: 'does not exist'


* Loading JSON data
* Looking up key


In [10]:
UNDEFINED = object()
DIE_IN_ELSE_BLOCK = False

def divide_json(path):
    print('* Opening file')
    handle = open(path, 'r+')   # May raise OSError
    try:
        print('* Reading data')
        data = handle.read()    # May raise UnicodeDecodeError
        print('* Loading JSON data')
        op = json.loads(data)   # May raise ValueError
        print('* Performing calculation')
        value = (
            op['numerator'] /
            op['denominator'])  # May raise ZeroDivisionError
    except ZeroDivisionError as e:
        print('* Handling ZeroDivisionError')
        return UNDEFINED
    else:
        print('* Writing calculation')
        op['result'] = value
        result = json.dumps(op)
        handle.seek(0)          # May raise OSError
        if DIE_IN_ELSE_BLOCK:
            import errno
            import os
            raise OSError(errno.ENOSPC, os.strerror(errno.ENOSPC))
        handle.write(result)    # May raise OSError
        return value
    finally:
        print('* Calling close()')
        handle.close()          # Always runs

In [11]:
temp_path = 'random_data.json'

In [12]:
with open(temp_path, 'w') as f:
    f.write('{"numerator": 1, "denominator": 10}')

assert divide_json(temp_path) == 0.1

* Opening file
* Reading data
* Loading JSON data
* Performing calculation
* Writing calculation
* Calling close()


In [13]:
with open(temp_path, 'w') as f:
    f.write('{"numerator": 1, "denominator": 0}')

assert divide_json(temp_path) is UNDEFINED

* Opening file
* Reading data
* Loading JSON data
* Performing calculation
* Handling ZeroDivisionError
* Calling close()


In [14]:
with open(temp_path, 'w') as f:
    f.write('{"numerator": 1 bad data')

try:
    divide_json(temp_path)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-14-f703666c83e2>", line 5, in <module>
    divide_json(temp_path)
  File "<ipython-input-10-9711f152fafb>", line 11, in divide_json
    op = json.loads(data)   # May raise ValueError
  File "/usr/local/lib/python3.8/json/__init__.py", line 357, in loads
    return _default_decoder.decode(s)
  File "/usr/local/lib/python3.8/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/usr/local/lib/python3.8/json/decoder.py", line 353, in raw_decode
    obj, end = self.scan_once(s, idx)
json.decoder.JSONDecodeError: Expecting ',' delimiter: line 1 column 17 (char 16)


* Opening file
* Reading data
* Loading JSON data
* Calling close()


In [15]:
DIE_IN_ELSE_BLOCK = True

with open(temp_path, 'w') as f:
    f.write('{"numerator": 1, "denominator": 10}')

try:
    divide_json(temp_path)
except:
    logging.exception('Expected')
else:
    assert False

ERROR:root:Expected
Traceback (most recent call last):
  File "<ipython-input-15-e5ecb888209e>", line 7, in <module>
    divide_json(temp_path)
  File "<ipython-input-10-9711f152fafb>", line 27, in divide_json
    raise OSError(errno.ENOSPC, os.strerror(errno.ENOSPC))
OSError: [Errno 28] No space left on device


* Opening file
* Reading data
* Loading JSON data
* Performing calculation
* Writing calculation
* Calling close()


> - `try`/`finally` 복합문을 사용하면 `try` 블록이 실행되는 동안 예외가 발생하든 발생하지 않든 정리 코드를 실행할 수 있다.
> - `else` 블록을 사용하면 `try` 블록 안에 넣을 코드를 최소화하고, `try`/`except` 블록과 성공적인 경우에 수행해야 할 코드를 시각적으로 구분할 수 있다.
> - `try` 블록이 성공적으로 처리되고 `finally` 블록이 공통적인 정리 작업을 수행하기 전에 실행해야 하는 동작이 있는 경우 `else` 블록을 사용할 수 있다.